In [ ]:
#@title
# Upload: 
# gan_faces_96.zip

# 8.7.1. Generative Adversarial Network (GAN)

In [ ]:
#@title 8.7.1. Import some necessary packages
import numpy as np
import pandas as pd
import glob
import time 

from sklearn.model_selection import train_test_split

import matplotlib
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models

# Search online about "ADAM" parameters
from keras.optimizers import Adam

from IPython.display import clear_output

In [ ]:
#@title 8.7.2. Data processing
# https://thispersondoesnotexist.com/ 
!unzip gan_faces_96.zip
batch_files = glob.glob('/content/*.npy')

datain = np.empty((len(batch_files) * 200,96,96,1))
c = 0
for i0 in range(len(batch_files)):
  datain[c:c+200] = np.load(batch_files[i0])
  c             = c + 200

datain.astype("float32")
datain = datain/255

# Observe some data
f, ax = plt.subplots(5, 5, sharex= True, sharey= True, figsize=(10,10))
f.subplots_adjust(hspace=0)
c = 0
for i0 in range(5):
  for i1 in range(5):
    ax[i0,i1].imshow(datain[c,:,:,0], cmap='gray')
    c = c+1


In [ ]:
#@title 8.7.3. Discriminator function
def fun_discriminator():
  model = models.Sequential()

  # Search online to learn about "LeakyReLU"

  model.add(layers.Conv2D(256, (3,3), strides=(2, 2), padding='same', input_shape=(96,96,1) ))
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Dropout(0.4))

  model.add(layers.Conv2D(256, (3,3), strides=(2, 2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Dropout(0.4))

  model.add(layers.Flatten())
  model.add(layers.Dense(1, activation='sigmoid'))

  # Search online about "ADAM" parameters

  opt = Adam(lr=0.00005, beta_1=0.5)

  model.compile(loss='mean_squared_error', optimizer=opt)

  return model

In [ ]:
#@title 8.7.4. Generator function
def fun_generator(hidden_size):
  model = models.Sequential()

  model.add(layers.Dense(128 * 16* 16, input_dim=hidden_size))
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Reshape((16, 16, 128)))

  model.add(layers.Conv2DTranspose(512, (3,3), strides=(2,2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))

  model.add(layers.Conv2DTranspose(512, (3,3), strides=(2,2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))

  model.add(layers.Conv2DTranspose(512, (3,3), strides=(2,2), padding='same'))
  model.add(layers.LeakyReLU(alpha=0.2))

  model.add(layers.Conv2D(1, (33,33), strides=(1,1), activation='sigmoid'))

  #model.summary()

  # We do not compile it. We need to add discriminator to the generator then compile it

  return model

In [ ]:
#@title 8.7.5. Gan function
def fun_gan(model_discriminator, model_generator):
  
  # We just want the generator parameter being updated each time we call model_gan
  model_discriminator.trainable = False

  model = models.Sequential()

  model.add(model_generator)
  model.add(model_discriminator)

  opt = Adam(lr=0.00005, beta_1=0.5)

  model.compile(loss='mean_squared_error', optimizer=opt)

  return model

In [ ]:
#@title 8.7.6. Useful functions
# Hidden generator (latent space value's generator)
def fun_latent(batch_size, hidden_size):
  hiddenin = np.random.randn(batch_size, hidden_size)
  hiddenpr = np.ones((batch_size,1))
  return hiddenin, hiddenpr

# real data generator 
def fun_real_generator(batch_size, data):
  ind    = np.random.permutation(data.shape[0])
  datain = data[ind[:batch_size],:]
  datapr = np.ones((datain.shape[0],1))
  return datain, datapr

# fake data generator 
def fun_fake_generator(model_generator, hiddenin):
  datain = model_generator.predict(hiddenin)
  datapr = np.zeros((datain.shape[0],1))
  return datain, datapr

# plot generator 
def fun_plot(data):

  shape0 = data.shape[0]
  shape1 = data.shape[1]
  shape2 = data.shape[2]
  sq_val = int(np.sqrt(shape0))

  image   = np.empty((sq_val*shape1, sq_val*shape2))
  counter = 0
  r = 0
  for j0 in range(sq_val):
    c = 0
    for j1 in range(sq_val):
      image[r:r+96,c:c+96] = data[counter,:,:]
      counter = counter + 1
      c = c + 96
    r = r + 96

  return image
  

In [ ]:
#@title 8.7.7. Training parameters and GAN models
batch_size  = 8
hidden_size = 32
epochs      = 50

num_batch   = int(datain.shape[0]/batch_size)

model_discriminator = fun_discriminator()
model_generator     = fun_generator(hidden_size)
model_gan           = fun_gan(model_discriminator, model_generator)

print("$$$$$$$$$$$$$$$$$$$$$$$$$")
print('Discriminator')
print("$$$$$$$$$$$$$$$$$$$$$$$$$")
model_discriminator.summary()

print("$$$$$$$$$$$$$$$$$$$$$$$$$")
print('Generator')
print("$$$$$$$$$$$$$$$$$$$$$$$$$")
model_generator.summary()

print("$$$$$$$$$$$$$$$$$$$$$$$$$")
print('GAN')
print("$$$$$$$$$$$$$$$$$$$$$$$$$")
model_gan.summary()

In [ ]:
#@title 8.7.8. Training
for i0 in range(epochs):
  for i1 in range(num_batch):
    hiddenin,_               = fun_latent(batch_size, hidden_size)

    datain_real, datapr_real = fun_real_generator(batch_size, datain) 
    datain_fake, datapr_fake = fun_fake_generator(model_generator, hiddenin) 

    datain_comb = np.concatenate((datain_real, datain_fake), axis = 0)
    datapr_comb = np.concatenate((datapr_real, datapr_fake), axis = 0)

    loss_discriminator_comb = model_discriminator.train_on_batch(datain_comb, datapr_comb)

    prob_discriminator_fake = model_discriminator.predict(datain_fake).mean()
    prob_discriminator_real = model_discriminator.predict(datain_real).mean()

    hiddenin, hiddenpr      = fun_latent(batch_size * 2, hidden_size)
    loss_generator          = model_gan.train_on_batch(hiddenin, hiddenpr)

    if i1%10  == 0:
      image    = fun_plot(model_generator.predict(hiddenin)[:,:,:,0])
      img_name = 'img_file_{:05d}_{:05d}.npy'.format(i0,i1) 
      np.save(img_name,image)

      clear_output(wait=True)
      plt.figure(figsize = [10,10])
      plt.imshow(image, cmap = 'gray')
      plt.title("Epoch#: {:03.0f}/{} | Batch#: {:04.0f}/{} | D-Loss: {:05.5f} | G-Loss: {:05.5f} | P-Real: {:0.7f} | P-Fake: {:0.7f} ".format(i0+1, epochs, i1, num_batch, loss_discriminator_comb, loss_generator,prob_discriminator_real,prob_discriminator_fake))
      plt.show()

In [ ]:
#@title 8.7.9. Generate some new images
hiddenin,_               = fun_latent(25, hidden_size)
datain_fake, datapr_fake = fun_fake_generator(model_generator, hiddenin) 

f, ax = plt.subplots(5, 5, sharex= True, sharey= True, figsize=(10,10))
f.subplots_adjust(hspace=0)
c = 0
for i0 in range(5):
  for i1 in range(5):
    ax[i0,i1].imshow(datain_fake[c,:,:,0], cmap='gray')
    c = c+1